Implementations of simple greedy vs brute force algorithms; used to demonstrate real run time

In [4]:
#Greedy algorithm

def addEdge(adjacent, vertex_1, vertex_2):
    adjacent[vertex_1].append(vertex_2) #adds vertex_2 to vertex_1s adjacencies
    adjacent[vertex_2].append(vertex_1) #adds vertex_1 to vertex_2s adjacencies

    return adjacent #returns adjacencies

def greedy(adjacent, num_vertices):
    colors = [-1] * num_vertices #create an array of length num_vertices filled with -1s; this array will hold the colors for each vertex
    available_colors = [False] * num_vertices #create an array of length num_vertices filled with False; this array will hold which colors are available for that vertex


    colors[0] = 0 #assign the first vertex to the first color

    for vertex in range(1, num_vertices): #loop through all of the vertices
        for adj_v in adjacent[vertex]: #loop through all of the adjacencies for that vertex
            if colors[adj_v] != -1: #if the color is not taken by any of the adjacent vertices
                available_colors[colors[adj_v]] = True #then it is an available color for this vertex
        
        lowest_color = 0
        
        while lowest_color < num_vertices:
            if available_colors[lowest_color] == False:
                break
            lowest_color += 1
        
        colors[vertex] = lowest_color
        available_colors = [False] * num_vertices # reset available colors for next loop

        # for adj_v in adjacent[vertex]: #loop back through the adjacenices
        #     if colors[adj_v] != -1: #find all of the values we changed to True earlier
        #         available_colors[colors[adj_v]] = False #change them back to false to reset the available_colors array

    for vertex in range(num_vertices):
        print(f"Vertex {vertex} is colored {colors[vertex]}") #print all of the vertices and their color

if __name__ == '__main__':
     
    g1 = [[] for _ in range(5)]
    g1 = addEdge(g1, 0, 1)
    g1 = addEdge(g1, 0, 2)
    g1 = addEdge(g1, 1, 2)
    g1 = addEdge(g1, 1, 3)
    g1 = addEdge(g1, 2, 3)
    g1 = addEdge(g1, 3, 4)
    greedy(g1, 5)

Vertex 0 is colored 0
Vertex 1 is colored 1
Vertex 2 is colored 2
Vertex 3 is colored 0
Vertex 4 is colored 1
